In [ ]:
import numpy as np
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pathlib
!pip install transformers
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
!pip install POT
from transformers import pipeline
!python -m spacy download en
import en_core_web_sm
nlp = en_core_web_sm.load()



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-23 22:51:40.402285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# gettting the answer files
student_files = [doc for doc in os.listdir(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')) if doc.endswith('.txt')]
student_names =[File[::-1][4:][::-1] for File in  student_files]
student_answers =[open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')/File,"r",encoding="utf-8").read() for File in  student_files]
student_names

['Student Answers 2',
 'Student Answers 1',
 'Student Answers 3',
 'Student Answers 4']

In [ ]:
from collections import defaultdict
plag_scores=defaultdict(list)
student_vectors=TfidfVectorizer().fit_transform(student_answers).toarray()
for i in range(len(student_names)-1):
  for j in range(i+1,len(student_names)):
    cs=cosine_similarity([student_vectors[i],student_vectors[j]])[0][1]
    if plag_scores[student_names[i]]:
      if plag_scores[student_names[i]][0][1]<=cs:
        plag_scores[student_names[i]].insert(0,[student_names[j],cs])
      else:
         plag_scores[student_names[i]].append([student_names[j],cs])
    else:
      plag_scores[student_names[i]].append([student_names[j],cs])
    if plag_scores[student_names[j]]:
      if plag_scores[student_names[j]][0][1]<=cs:
        plag_scores[student_names[j]].insert(0,[student_names[i],cs])
      else:
         plag_scores[student_names[j]].append([student_names[i],cs])
    else:
      plag_scores[student_names[j]].append([student_names[i],cs])
for i,j in plag_scores.items():
  print("Highest plagarism percentage of",i,"found with ",j[0][0],"-",j[0][1]*100 )
  

Highest plagarism percentage of Student Answers 2 found with  Student Answers 4 - 80.57355872855332
Highest plagarism percentage of Student Answers 1 found with  Student Answers 2 - 74.18991155821108
Highest plagarism percentage of Student Answers 3 found with  Student Answers 2 - 71.23290924300537
Highest plagarism percentage of Student Answers 4 found with  Student Answers 2 - 80.57355872855332


In [ ]:
!pip install language-tool-python
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


INFO:language_tool_python.download_lt:Unzipping /tmp/tmpgir6r0bh.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


In [ ]:
#grammarcheck
gramatical_scores=[]
Answers=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')/'Student Answers 4.txt',"r",encoding="utf-8").read().split('Answer—')
print(Answers)
for answer in Answers:
    matches = tool.check(answer)
    gramatical_scores.append(matches)
gramatical_scores







  


['\ufeff1.Decision trees are non-linear classifiers because they can simulate intricate decision limits that are impossible for linear classifiers to do. Up until the data is categorized into pure classes, the tree recursively separates the feature space based on the input features. According to the input features, each split in the tree corresponds to a decision boundary, which can have any shape. Decision trees are hence non-linear classifiers.\n\n\n\n\n\n\n\n\n', '\n\n\n\n\n2. In contrast to bagging, which trains models simultaneously, boosting trains models sequentially in order to increase the performance of the entire ensemble.\n\n\n\n\n\n\n', '\n\n\n\n\n\n\n\n\n3. A suffix array is a collection of all the suffixes of a given string, arranged in alphabetical order.\n\n\n\n\n\n\n\n\n', '\n\n\n\n\n4.The act of establishing a new class that is based on an existing class is known as inheritance. The new class has the option to both override and inherit the properties and methods of t

[[Match({'ruleId': 'SENTENCE_WHITESPACE', 'message': 'Add a space between sentences.', 'replacements': [' Decision'], 'offsetInContext': 3, 'context': '\ufeff1.Decision trees are non-linear classifiers becaus...', 'offset': 3, 'errorLength': 8, 'category': 'TYPOGRAPHY', 'ruleIssueType': 'whitespace', 'sentence': 'Decision trees are non-linear classifiers because they can simulate intricate decision limits that are impossible for linear classifiers to do.'})],
 [],
 [],
 [Match({'ruleId': 'SENTENCE_WHITESPACE', 'message': 'Add a space between sentences.', 'replacements': [' The'], 'offsetInContext': 7, 'context': '     4.The act of establishing a new class that is...', 'offset': 7, 'errorLength': 3, 'category': 'TYPOGRAPHY', 'ruleIssueType': 'whitespace', 'sentence': 'The act of establishing a new class that is based on an existing class is known as inheritance.'})]]

In [ ]:
from collections import defaultdict
questions_keywords=defaultdict(list)
questions_keywords={'Que1':[["Naive Bayes","classification"," sentiment prediction","text classification"],["Syntactic parsing","syntactic structure","identifyin"],["shortening","summary","Extraction-based summarization","Abstraction-based summarization"],["word frequencies","matrix"]]}


In [ ]:
bert_model = SentenceTransformer('stsb-roberta-large')
def sim_score(solutions,answers,model):
  ans_scores=[]
  Solutions=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Solutions')/solutions,"r",encoding="utf-8").read().split('Answer—')
  Answers=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')/answers,"r",encoding="utf-8").read().split('Answer—')
  if len(Answers)!=len(Solutions):
    Answers.append('')
  scores=[]
  for i in range(len(Solutions)):
    c_s={}
    c_a={}
    sol_sentence_tokens=Solutions[i].split('.')[:-1]
    for j in range(len(sol_sentence_tokens)):
        sen_keywords=nlp(sol_sentence_tokens[j])  
        c_s[j]=list(sen_keywords)
    ans_sentence_tokens=Answers[i].split('.')[:-1]
    for j in range(len(ans_sentence_tokens)):
      sen_keywords=nlp(ans_sentence_tokens[j])
      c_a[j]=list(sen_keywords)
    comp_scores=[]
    for s in range(len(sol_sentence_tokens)):
      max_score=0
      for a in range(len(ans_sentence_tokens)):
        k=len(set(c_s[s])&set(c_a[a]))/len(c_s[s])
        if k<0.3:
          k=0
        emb1=model.encode(sol_sentence_tokens[s], convert_to_tensor=True)
        emb2=model.encode(ans_sentence_tokens[a], convert_to_tensor=True)
        cosine_scores = util.pytorch_cos_sim(emb1, emb2)
        max_score=max(k+cosine_scores.item(),max_score)
      comp_scores.append(max_score)
    ans_scores.append(sum(comp_scores)/len(sol_sentence_tokens))
  return ans_scores
      # model = SentenceTransformer('stsb-roberta-large')
    # embedding1 = model.encode(sentence1, convert_to_tensor=True)
    # embedding2 = model.encode(sentence2, convert_to_tensor=True)

In [ ]:
import gensim.downloader as api
word2vecmodel = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
def getWord_movers_distance(solutions,answers):
  ans_scores=[]
  # model = SentenceTransformer('stsb-roberta-large')
  Solutions=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Solutions')/solutions,"r",encoding="utf-8").read().split('Answer—')
  Answers=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')/answers,"r",encoding="utf-8").read().split('Answer—')
  if len(Answers)!=len(Solutions):
    Answers.append('')
  scores=[]
  for i in range(len(Solutions)):
    c_s={}
    c_a={}
    sol_sentence_tokens=Solutions[i].split('.')[:-1]
    for j in range(len(sol_sentence_tokens)):
        sen_keywords=nlp(sol_sentence_tokens[j])
        c_s[j]=list(sen_keywords)
    ans_sentence_tokens=Answers[i].split('.')[:-1]
    for j in range(len(ans_sentence_tokens)):
      sen_keywords=nlp(ans_sentence_tokens[j])
      c_a[j]=list(sen_keywords)
    comp_scores=[]
    for s in range(len(sol_sentence_tokens)):
      max_score=0
      for a in range(len(ans_sentence_tokens)):
        k=len(set(c_s[s])&set(c_a[a]))/len(c_s[s])
        wdm=word2vecmodel.wmdistance(sol_sentence_tokens[s],ans_sentence_tokens[a])
        if wdm<=0.7:
          wdm=1-wdm
        elif wdm<1.6:
          wdm=1.6-wdm
        max_score=max(k+wdm,max_score)
      comp_scores.append(max_score)
    ans_scores.append(sum(comp_scores)/len(sol_sentence_tokens))
  return ans_scores
        

def getcosinesimilarity_word2vec(solutions,answers):
  ans_scores=[]
  # model = SentenceTransformer('stsb-roberta-large')
  Solutions=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Solutions')/solutions,"r",encoding="utf-8").read().split('Answer—')
  Answers=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')/answers,"r",encoding="utf-8").read().split('Answer—')
  if len(Answers)!=len(Solutions):
    Answers.append('')
  scores=[]
  for i in range(len(Solutions)):
    c_s={}
    c_a={}
    sol_sentence_tokens=Solutions[i].split('.')[:-1]
    for j in range(len(sol_sentence_tokens)):
        sen_keywords=nlp(sol_sentence_tokens[j])
        c_s[j]=list(sen_keywords)
    ans_sentence_tokens=Answers[i].split('.')[:-1]
    for j in range(len(ans_sentence_tokens)):
      sen_keywords=nlp(ans_sentence_tokens[j])
      c_a[j]=list(sen_keywords)
    comp_scores=[]
    for s in range(len(sol_sentence_tokens)):
      max_score=0
      for a in range(len(ans_sentence_tokens)):
        k=len(set(c_s[s])&set(c_a[a]))/len(c_s[s])
        cosine=word2vecmodel.n_similarity(sol_sentence_tokens[s].lower().split(), ans_sentence_tokens[a].lower().split())
        max_score=max(k+cosine,max_score)
      comp_scores.append(max_score)
    ans_scores.append(sum(comp_scores)/len(sol_sentence_tokens))
  return ans_scores    





In [ ]:
!pip install "tensorflow>=2.0.0"
!pip install --upgrade tensorflow-hub
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
def elmo_vectors(x):
  embeddings=elmo(x, signature="default", as_dict=True)["elmo"]
  with tf.device('/device:GPU:0'):
    with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      sess.run(tf.tables_initializer())
      return sess.run(tf.reduce_mean(embeddings,1))

def elmo_sim_score(solutions,answers):
  ans_scores=[]
  Solutions=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Solutions')/solutions,"r",encoding="utf-8").read().split('Answer—')
  Answers=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')/answers,"r",encoding="utf-8").read().split('Answer—')
  if len(Answers)!=len(Solutions):
    Answers.append('')
  scores=[]
  for i in range(len(Solutions)):
    c_s={}
    c_a={}
    sol_sentence_tokens=Solutions[i].split('.')[:-1]
    for j in range(len(sol_sentence_tokens)):
        sen_keywords=nlp(sol_sentence_tokens[j])  
        c_s[j]=list(sen_keywords)
    ans_sentence_tokens=Answers[i].split('.')[:-1]
    for j in range(len(ans_sentence_tokens)):
      sen_keywords=nlp(ans_sentence_tokens[j])
      c_a[j]=list(sen_keywords)
    comp_scores=[]
    for s in range(len(sol_sentence_tokens)):
      max_score=0
      for a in range(len(ans_sentence_tokens)):
        k=len(set(c_s[s])&set(c_a[a]))/len(c_s[s])
        if k<0.3:
          k=0
        sens=[sol_sentence_tokens[s], ans_sentence_tokens[a]]
        elmo_embeddings=[]
        for i in range(2):
           elmo_embeddings.append(elmo_vectors([sens[i]])[0])
        similarities = cosine_similarity(elmo_embeddings, elmo_embeddings)
        max_score=max(k+similarities[0][1],max_score)
      comp_scores.append(max_score)
    ans_scores.append(sum(comp_scores)/len(sol_sentence_tokens))
  return ans_scores

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
print("Cosine similarity with bert embeddings on an answer sheet and solution",sim_score('Solution.txt','Student Answers 4.txt',bert_model))
print("Word movers distance similarity with word2vec embeddings on an answer sheet and solution",getWord_movers_distance('Solution.txt','Student Answers 4.txt'))
print("Cosine similarity with Word2Vec embeddings on an answer sheet and solution",getcosinesimilarity_word2vec('Solution.txt','Student Answers 4.txt'))
print("Cosine similarity with Elmo embeddings on an answer sheet and solution",elmo_sim_score('Solution.txt','Student Answers 4.txt'))

Cosine similarity with Elmo embeddings on an answer sheet and solution [0.8116208712259928, 0.6611632903416952, 0.7175295948982239, 0.6245733797550201]


In [ ]:
model = SentenceTransformer('stsb-roberta-large')
emb1=model.encode("We went to bank of the river", convert_to_tensor=True)
emb2=model.encode("We went to bank to deposit money", convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(emb1, emb2)
cosine_w2vec=word2vecmodel.n_similarity("We went to bank of the river".lower().split(), "We went to bank to deposit money".lower().split())

In [ ]:
print("Cosine_similarity with bert embeddings",cosine_scores.item())
print("Cosine_similarity with Word2Vec embeddings",cosine_w2vec)

Cosine_similarity with bert embeddings 0.5395205616950989
Cosine_similarity with Word2Vec embeddings 0.74597085


In [ ]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP Project/Train - Sheet1.csv")
data.head()

,Original_Answer,Student_Answer,Grade
0,A decision tree is a non-linear classifier bec...,Decision tree is a non-linear classifier becau...,A
1,A decision tree is a non-linear classifier bec...,Decision tree is a non-linear classifier becau...,A
2,A decision tree is a non-linear classifier bec...,Decision tree is a non-linear classifier becau...,B
3,A decision tree is a non-linear classifier bec...,Decision tree is a non-linear classifier becau...,B
4,A decision tree is a non-linear classifier bec...,Decision tree is a non-linear classifier becau...,C


In [ ]:
data['pre_process_answer'] = data['Student_Answer'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['pre_process_solution'] = data['Original_Answer'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
data['pre_process_answer']=data['pre_process_answer'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))
data['pre_process_solution']=data['pre_process_solution'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
data['pre_process_answer']=data['pre_process_answer'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))
data['pre_process_solution']=data['pre_process_solution'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,Original_Answer,Student_Answer,Grade,pre_process_answer,pre_process_solution
0,A decision tree is a non-linear classifier bec...,Decision tree is a non-linear classifier becau...,A,decision tree non-linear classifier partition ...,decision tree non-linear classifier us hierarc...
1,A decision tree is a non-linear classifier bec...,Decision tree is a non-linear classifier becau...,A,decision tree non-linear classifier model comp...,decision tree non-linear classifier us hierarc...
2,A decision tree is a non-linear classifier bec...,Decision tree is a non-linear classifier becau...,B,decision tree non-linear classifier partition ...,decision tree non-linear classifier us hierarc...
3,A decision tree is a non-linear classifier bec...,Decision tree is a non-linear classifier becau...,B,decision tree non-linear classifier capture no...,decision tree non-linear classifier us hierarc...
4,A decision tree is a non-linear classifier bec...,Decision tree is a non-linear classifier becau...,C,decision tree non-linear classifier capture co...,decision tree non-linear classifier us hierarc...


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
data.head()
d=vectorizer.fit_transform(data.iloc[:, 3:4].apply(lambda x: ' '.join(x), axis=1))
X_train,X_test,Y_train, Y_test = train_test_split(d, data['Grade'], test_size=0.05, random_state=30)

In [ ]:
from sklearn.svm import LinearSVC
Linear_svc = LinearSVC(random_state=0)
Linear_svc.fit(X_train,Y_train)

LinearSVC(random_state=0)

In [ ]:
y_pred = Linear_svc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.36363636363636365


In [ ]:
def preprocess(answer):
  answer.lower()
  s=""
  for word in answer.split():
    if word not in stop:
      s=s+" "+word
  ls=""
  for word in nltk.word_tokenize(s):
    ls=ls+" "+lemmatizer.lemmatize(word)
  return ls

  



In [ ]:
Answers=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')/'Student Answers 4.txt',"r",encoding="utf-8").read().split('Answer—')
for i in Answers:
   print(Linear_svc.predict(vectorizer.transform([preprocess(i)])))

['A']
['B']
['B']
['C']


In [ ]:
from sklearn.naive_bayes import MultinomialNB
Nb = MultinomialNB(force_alpha=True)
Nb.fit(X_train,Y_train)
y_pred = Linear_svc.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy: {accuracy}")
Answers=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')/'Student Answers 4.txt',"r",encoding="utf-8").read().split('Answer—')
for i in Answers:
   print(Nb.predict(vectorizer.transform([preprocess(i)])))

Accuracy: 0.36363636363636365
['A']
['B']
['B']
['A']


In [ ]:
def Final_score_module(solution):
   student_files = [doc for doc in os.listdir(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')) if doc.endswith('.txt')]
   student_names =[File for File in  student_files]
   bert_model = SentenceTransformer('stsb-roberta-large')
   for student in student_names:
     cosine_simscores=sim_score(solution,student,bert_model)
     word_movers_distance=getWord_movers_distance(solution,student)
     scores=[]
     for i in range(len(cosine_simscores)):
       scores.append(max(cosine_simscores[i],word_movers_distance[i])) 
     Answers=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')/student,"r",encoding="utf-8").read().split('Answer—')
     Model_scores=[]
     for i in Answers:
            Model_scores.append(clf.predict(vectorizer.transform([preprocess(i)])))
     for i in range(len(scores)):
      scores[i]=scores[i]*100
     for i in range(len(scores)):
      if scores[i]>=90:
        if Model_scores[i]=='B':
          scores[i]=scores[i]-5
        elif Model_scores[i]=='C':
          scores[i]=scores[i]-7.5
        elif Model_scores[i]=='D':
          scores[i]=scores[i]-10
      elif scores[i]>=80:
        if Model_scores[i]=='A':
          scores[i]=scores[i]+5
        elif Model_scores[i]=='C':
          scores[i]=scores[i]-5
        elif Model_scores[i]=='D':
          scores[i]=scores[i]-10
      elif scores[i]>=70:
        if Model_scores[i]=='A':
          scores[i]=scores[i]+7.5
        elif Model_scores[i]=='B':
          scores[i]=scores[i]+5
        elif Model_scores[i]=='D':
          scores[i]=scores[i]-5
      else:
        if Model_scores[i]=='A':
          scores[i]=scores[i]+10
        elif Model_scores[i]=='B':
          scores[i]=scores[i]+7.5
        elif Model_scores[i]=='C':
          scores[i]=scores[i]+5
     print(scores)
    
Final_score_module('Solution.txt')



    





[79.49118490980227, 77.77705637308406, 90.59341188459148, 89.35274961284307]
[75.3254389571375, 94.99527858823822, 77.73648995705071, 80.05087380903498]
[78.61774966406668, 79.60763509131137, 73.85395727813308, 75.26155592099961]
[91.27349989859118, 85.95865839785685, 78.79520843823418, 79.23275983875817]


In [ ]:
import torch
!pip install transformers
!pip install sentencepiece
from transformers import AutoTokenizer, XLNetForSequenceClassification,TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
import numpy as np

In [ ]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP Project/Train - Sheet1.csv")
data.head()

In [ ]:
Grades=[]
student_answerslist=data.Student_Answer.values.tolist()
for i in data.Grade.values.tolist():
  if i=="A":
    Grades.append([1,0,0,0])
  elif i=="B":
    Grades.append([0,1,0,0])
  elif i=="C":
    Grades.append([0,0,1,0])
  else:
    Grades.append([0,0,0,1])
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
!pip install evaluate
import evaluate
X_train, X_test, y_train, y_test = train_test_split(student_answerslist,Grades, test_size=0.2, random_state=0)
train_tokenized = tokenizer(X_train, truncation=True, padding=True)
test_tokenized = tokenizer(X_test, truncation=True, padding=True)
class torchdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, index):
        item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[index],dtype=torch.float32)
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = torchdataset(train_tokenized, y_train)
test_dataset = torchdataset(test_tokenized, y_test)
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    print("logits shape:", logits[0].shape)
    print("labels shape:", labels.shape)
    predictions=[]
    for i in range(len(logits)):
      if np.argmax(logits[i])==0:
        predictions.append(0)
      elif np.argmax(logits[i])==1:
        predictions.append(1)
      elif np.argmax(logits[i])==0:
        predictions.append(2)
      else:
        predictions.append(3)
    predictions=np.array(predictions)
    references=[]
    for i in range(len(labels)):
      if np.argmax(labels[i])==0:
        references.append(0)
      elif np.argmax(labels[i])==1:
        references.append(1)
      elif np.argmax(labels[i])==0:
        references.append(2)
      else:
        references.append(3)
    references=np.array(references)
    print("predictions shape:", predictions.shape)
    print("labels shape:", labels.shape)
    return metric.compute(predictions=predictions, references=references)


In [ ]:
from collections import Counter
np.array(test_dataset).shape


In [ ]:
xlnet_model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", problem_type="multi_label_classification",num_labels=4)
xlnet_model.classifier = torch.nn.Linear(768,4)
print(xlnet_model.sequence_summary)
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=3)
xlnet_trainer = Trainer(model=xlnet_model,args=training_args,train_dataset=train_dataset,eval_dataset=test_dataset,compute_metrics=compute_metrics)

In [ ]:
xlnet_trainer.train()

In [ ]:

import pathlib
Answers=open(pathlib.Path('/content/drive/MyDrive/Colab Notebooks/NLP Project/Students')/'Student Answers 4.txt',"r",encoding="utf-8").read().split('Answer—')
text_tokenized=tokenizer(Answers, truncation=True, padding=True)
tdata=torchdataset(text_tokenized, [[0,0,0,0]]*len(Answers))
xlnet_trainer.predict(tdata)